# Получение и первичная обработка данных

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Activation


def get_data():
    con = sqlite3.connect("wikibooks.sqlite")
    cur = con.cursor()
    res = cur.execute("SELECT body_text FROM ru")
    all_texts = res.fetchall()
    return pd.DataFrame({"text": [item[0] for item in all_texts]})

def contains_non_russian(text):
    russian_alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя, .!')
    words = text.split()
    for word in words:
        if any(char.lower() not in russian_alphabet for char in word):
            return True
    return False

def filter_non_russian_strings(strings):
    return [string.lower() for string in strings if not contains_non_russian(string)]

# Чтение строк из файла
with open("dataset.txt", 'rb') as file:
    lines = []
    for line in file:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)

# **Simple RNN с посимвольной и по-словной токенизацией**

Simple RNN (рекуррентная нейронная сеть) - это тип нейронных сетей, который может обрабатывать последовательности данных, такие как текст, звук или временные ряды. Он имеет обратную связь, позволяющую использовать предыдущие выходные данные для обработки текущего входа.

Посимвольная токенизация означает разбиение текста на отдельные символы, а по-словной токенизации - на отдельные слова. Оба подхода могут быть использованы для обработки текста Simple RNN.

Давайте рассмотрим примеры кода для посимвольной и по-словной токенизации с использованием библиотеки Python Keras:

### Посимвольная токенизация

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical

# Объединение всех строк в одну
text = " ".join(filter_non_russian_strings(lines)[:10000])

# Создание множества уникальных символов
chars = set([c for c in text])

# Количество уникальных символов
nb_chars = len(chars)

# Создание словарей для преобразования символов в числовые индексы и обратно
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# Длина последовательности символов и шаг для создания подпоследовательностей
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# Создание подпоследовательностей длиной SEQLEN и соответствующих символов-меток
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

# Преобразование последовательностей символов в матрицы one-hot кодирования
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.float32)
y = np.zeros((len(input_chars),), dtype=np.int32)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1.0
    y[i] = char2index[label_chars[i]]

# Задание параметров модели и обучения
NUM_ITERATIONS = 25
NUM_PREDS_PER_EPOCH = 100

# Создание модели рекуррентной нейронной сети
model = Sequential()
model.add(SimpleRNN(units=500))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop") 

In [20]:
model.fit(X, y, batch_size=128, epochs=5, verbose=1)

Epoch 1/5
3483/3483 [==============================] - 33s 9ms/step - loss: 1.6798
Epoch 2/5
3483/3483 [==============================] - 33s 10ms/step - loss: 1.6659
Epoch 3/5
3483/3483 [==============================] - 34s 10ms/step - loss: 1.6531
Epoch 4/5
3483/3483 [==============================] - 32s 9ms/step - loss: 1.6459
Epoch 5/5
3483/3483 [==============================] - 33s 10ms/step - loss: 1.6430


In [25]:
test_idx = 0
test_chars = input_chars[test_idx]

print("Генерация из начального символа: %s" % (test_chars))
print(test_chars, end="")
for i in range(NUM_PREDS_PER_EPOCH):
    X_test = np.zeros((1, SEQLEN, nb_chars))
    for j, ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1
    pred = model.predict(X_test, verbose=0)[0]
    # Следующий символ выбирается с учетом вероятностного распределения
    next_index = np.random.choice(nb_chars, p=pred)
    next_char = index2char[next_index]
    print(next_char, end="")
    # Обновление test_chars для следующей итерации
    test_chars = test_chars[1:] + next_char
print()

Генерация из начального символа: содержание
содержание пунш способрои описан импонтов и метиловностью предычетов при грандкой фина незы в от гоморных виде



## По-словная токенизация

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical

# Объединение всех строк в одну
texts = filter_non_russian_strings(lines)[:1000]

# Создание экземпляра класса Tokenizer
tokenizer = Tokenizer()

# Обучение токенизатора на массиве строк
tokenizer.fit_on_texts(texts)

# Преобразование строк в последовательности числовых индексов
sequences = tokenizer.texts_to_sequences(texts)

# Преобразование последовательностей входных и выходных данных
input_sequences = []
output_sequences = []
for seq in sequences:
    for i in range(1, len(seq)):
        input_sequence = seq[:i]
        output_sequence = seq[i]
        input_sequences.append(input_sequence)
        output_sequences.append(output_sequence)
    
# Заполнение последовательностей до одной и той же длины
input_sequences = list(input_sequences)
output_sequences = list(output_sequences)

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)

# Преобразование выходных последовательностей в one-hot кодировку
output_sequences = to_categorical(output_sequences, num_classes=len(tokenizer.word_index) + 1)


# Создание модели
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length))
model.add(SimpleRNN(units=128))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Обучение модели
model.fit(input_sequences, output_sequences, epochs=15, verbose=1)

Epoch 1/15
108/108 [==============================] - 22s 183ms/step - loss: 7.5125
Epoch 2/15
108/108 [==============================] - 12s 111ms/step - loss: 6.9080
Epoch 3/15
108/108 [==============================] - 10s 95ms/step - loss: 6.5360
Epoch 4/15
108/108 [==============================] - 9s 85ms/step - loss: 6.1986
Epoch 5/15
108/108 [==============================] - 7s 67ms/step - loss: 5.7274
Epoch 6/15
108/108 [==============================] - 8s 73ms/step - loss: 5.2437
Epoch 7/15
108/108 [==============================] - 7s 63ms/step - loss: 4.7716
Epoch 8/15
108/108 [==============================] - 8s 74ms/step - loss: 4.3037
Epoch 9/15
108/108 [==============================] - 8s 78ms/step - loss: 3.8403
Epoch 10/15
108/108 [==============================] - 7s 66ms/step - loss: 3.3855
Epoch 11/15
108/108 [==============================] - 8s 74ms/step - loss: 2.9509
Epoch 12/15
108/108 [==============================] - 7s 66ms/step - loss: 2.5380
Epoch 13

In [ ]:
# Предсказание продолжения строки
input_text_arr = ["можно получить", "для того чтобы", "в россии", "если площадь", "выполнение", "в современном", "хронический", "коррекция"]
for input_text in input_text_arr:
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length)
    predicted_sequence = model.predict(input_sequence)

    # Декодирование предсказанной последовательности в текст
    predicted_index = np.argmax(predicted_sequence, axis=-1)[0]
    predicted_text = tokenizer.sequences_to_texts([[predicted_index]])
    print(f"Сгенерированный текст: {input_text, predicted_text}")


1/1 [==============================] - 0s 133ms/step
Сгенерированный текст: ('можно получить', ['свидетельство'])
1/1 [==============================] - 0s 24ms/step
Сгенерированный текст: ('для того чтобы', ['развить'])
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('в россии', ['не'])
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('если площадь', ['параллелограмма'])
1/1 [==============================] - 0s 21ms/step
Сгенерированный текст: ('выполнение', ['ягоды'])
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('в современном', ['обществе'])
1/1 [==============================] - 0s 20ms/step
Сгенерированный текст: ('хронический', ['все'])
1/1 [==============================] - 0s 23ms/step
Сгенерированный текст: ('коррекция', ['эстетика'])


Оба примера создают Simple RNN модель с одним слоем SimpleRNN и выходным слоем Dense. Разница заключается в способе токенизации текста. 

После токенизации текст преобразуется в последовательность чисел, а затем каждая последовательность дополняется до одной длины, чтобы их можно было использовать в качестве входных данных для Simple RNN.

В посимвольном примере используется Tokenizer с параметром char_level=True, чтобы токенизировать текст на отдельные символы. Длина каждой последовательности установлена на maxlen=100. Входной слой SimpleRNN использует эту длину и количество уникальных символов в тексте, чтобы создать соответствующую форму входных данных.

В по-словном примере используется стандартный Tokenizer, который токенизирует текст на отдельные слова. Для обработки текста в этом случае используется слой Embedding, который преобразует числа в векторы фиксированной длины. Размер словаря (vocab_size) установлен на количество уникальных слов в тексте + 1 (для учета пустого токена). Длина входных последовательностей также установлена на maxlen=100.

# **2. Однонаправленная однослойная и многослойная LSTM c посимвольной токенизацией и токенизацией по словам и на основе BPE**

Выясним наименьшую/наибольшую и среднюю длины строк в нашем массиве предложений

In [ ]:
text_array = filter_non_russian_strings(lines)
# Вычисление максимальной длины строки
max_length = max(len(text) for text in text_array)

# Вычисление минимальной длины строки
min_length = min(len(text) for text in text_array)

# Вычисление средней длины строки
avg_length = sum(len(text) for text in text_array) / len(text_array)

print("Максимальная длина строки:", max_length)
print("Минимальная длина строки:", min_length)
print("Средняя длина строки:", avg_length)

Максимальная длина строки: 1132
Минимальная длина строки: 1
Средняя длина строки: 39.692088671954444


Удаляем тексты слишком длинные/слишком короткие

In [ ]:
# Отфильтруем только строки, которые содержат только русские символы
text_array = filter_non_russian_strings(lines)  
min_length = 35  # Минимальная длина строки, которую нужно сохранить
max_length = 200  # Минимальная длина строки, которую нужно сохранить

text_corpus = [text for text in text_array if len(text) >= min_length and len(text) <= max_length]
len(text_corpus)

1772

**Однонаправленная однослойная LSTM с посимвольной токенизацией:**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Создаем токенизатор для посимвольной токенизации
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(text_corpus)

# Установим значение атрибута num_words
tokenizer.num_words = len(tokenizer.word_index) + 1

# Преобразуем текст в числовой формат
sequences = tokenizer.texts_to_sequences(text_corpus)

# Подготавливаем данные для обучения
padded_sequences = pad_sequences(sequences)
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Создаем модель
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], 1)),
    Dense(tokenizer.num_words, activation='softmax')
])

# Компилируем модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
print(text_corpus[:100])

['также можно получить свидетельство за рубежом и для того чтобы использовать его в россии пройти процедуру валидации.', 'пилотские в россии не всегда были бессрочными. раньше свидетельство требовалось продлевать.', 'если площадь параллелограмма равна нулю, то векторы коллинеарны.', 'динамические структуры данныхгигиенические основы и медицинский контроль за физическим воспитанием школьников', 'влейте в шейкер джин, лимонный сок, абсент и яичный белок.', 'дважды процедите коктейль через ситечко в охлажденный коктейльный бокал.', 'дайте пене подняться, а потом аккуратно влейте несколько капель биттера.', 'бокал доверху наполняем кубиками льда.', 'поверх льда наливаем крепкий алкоголь и ананасовый фреш.', 'слегка все размешиваем, после чего добавляем фреш лимона.', 'в качестве украшения используем мякоть свежего ананаса.', 'используя коктейльную ложку, уложи слоями ликер мараскино, лаймовый сок и сверхкрепкий ром', 'молоко налейте в кастрюлю, хорошо подогрейте и снимите с плиты.', 'раств

In [ ]:
# Обучаем модель
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
56/56 [==============================] - 4s 10ms/step - loss: 2.7034 - accuracy: 0.3668
Epoch 2/10
56/56 [==============================] - 0s 8ms/step - loss: 2.2396 - accuracy: 0.4114
Epoch 3/10
56/56 [==============================] - 0s 8ms/step - loss: 2.1893 - accuracy: 0.4114
Epoch 4/10
56/56 [==============================] - 0s 7ms/step - loss: 2.1476 - accuracy: 0.4114
Epoch 5/10
56/56 [==============================] - 0s 8ms/step - loss: 2.1080 - accuracy: 0.4114
Epoch 6/10
56/56 [==============================] - 0s 7ms/step - loss: 2.0722 - accuracy: 0.4114
Epoch 7/10
56/56 [==============================] - 0s 7ms/step - loss: 2.0422 - accuracy: 0.4131
Epoch 8/10
56/56 [==============================] - 0s 8ms/step - loss: 2.0136 - accuracy: 0.4272
Epoch 9/10
56/56 [==============================] - 1s 10ms/step - loss: 1.9890 - accuracy: 0.4430
Epoch 10/10
56/56 [==============================] - 0s 8ms/step - loss: 1.9681 - accuracy: 0.4503


In [ ]:
# Генерируем текст
seed_text_arr = ["можно получить", "для того чтобы", "в россии", "если площадь", "выполнение", "в современном", "хронический", "коррекция"]
for input_text in seed_text_arr:
    input_sequence = tokenizer.texts_to_sequences(input_text)
    input_sequence = pad_sequences(input_sequence, maxlen=model.input_shape[1])
    predicted_sequence = model.predict(input_sequence)

    # Декодирование предсказанной последовательности в текст
    predicted_indices = predicted_sequence.argmax(axis=-1)
    predicted_text = ''.join([tokenizer.index_word[idx] for idx in predicted_indices])
    generated_text = predicted_text[:10]  # Получаем первые 10 символов

    # Соединение исходного и сгенерированного текста
    output_text = input_text + generated_text
    print(output_text)

1/1 [==============================] - 0s 21ms/step
можно получить.оу.оя.о..
1/1 [==============================] - 0s 21ms/step
для того чтобы...я.оуоя.
1/1 [==============================] - 0s 18ms/step
в россии.я.о..аа
1/1 [==============================] - 0s 19ms/step
если площадьа..ая..оуа
1/1 [==============================] - 0s 23ms/step
выполнение...о..а.аа
1/1 [==============================] - 0s 23ms/step
в современном.я.о..а.а.
1/1 [==============================] - 0s 20ms/step
хроническийу.о.а.а..а
1/1 [==============================] - 0s 23ms/step
коррекция.о..а.уа.


**Однонаправленная однослойная LSTM с токенизацией по словам:**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Создаем токенизатор для токенизации по словам
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_corpus)

# Преобразуем текст в числовой формат
sequences = tokenizer.texts_to_sequences(text_corpus)

# Установим значение атрибута num_words
tokenizer.num_words = len(tokenizer.word_index) + 1

# Подготавливаем данные для обучения
padded_sequences = pad_sequences(sequences)
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Создаем модель
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], 1)),
    Dense(tokenizer.num_words, activation='softmax')
])

# Компилируем модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Обучаем модель ~60 эпох
model.fit(X, y, epochs=15, verbose=1)

Epoch 1/15
56/56 [==============================] - 3s 7ms/step - loss: 8.2740 - accuracy: 0.0248
Epoch 2/15
56/56 [==============================] - 0s 5ms/step - loss: 6.6497 - accuracy: 0.0435
Epoch 3/15
56/56 [==============================] - 0s 5ms/step - loss: 6.1948 - accuracy: 0.0508
Epoch 4/15
56/56 [==============================] - 0s 5ms/step - loss: 5.9293 - accuracy: 0.0666
Epoch 5/15
56/56 [==============================] - 0s 5ms/step - loss: 5.7049 - accuracy: 0.0655
Epoch 6/15
56/56 [==============================] - 0s 5ms/step - loss: 5.4714 - accuracy: 0.0767
Epoch 7/15
56/56 [==============================] - 0s 5ms/step - loss: 5.2564 - accuracy: 0.0875
Epoch 8/15
56/56 [==============================] - 0s 5ms/step - loss: 5.0614 - accuracy: 0.0976
Epoch 9/15
56/56 [==============================] - 0s 5ms/step - loss: 4.8548 - accuracy: 0.1117
Epoch 10/15
56/56 [==============================] - 0s 5ms/step - loss: 4.6847 - accuracy: 0.1304
Epoch 11/15
56/56 [

In [ ]:
# Предсказание продолжения строки
input_text_arr = ["можно получить", "для того чтобы", "в россии", "если площадь", "выполнение", "в современном", "хронический", "коррекция"]
for input_text in input_text_arr:
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=len(padded_sequences[0])-1)
    predicted_sequence = model.predict(input_sequence)

    # Декодирование предсказанной последовательности в текст
    predicted_text = tokenizer.sequences_to_texts([[idx] for idx in predicted_sequence.argmax(axis=-1)])
    print(f"Сгенерированный текст: {input_text, predicted_text[0]}")

1/1 [==============================] - 0s 331ms/step
Сгенерированный текст: ('можно получить', 'преобразования')
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('для того чтобы', 'доверху')
1/1 [==============================] - 0s 20ms/step
Сгенерированный текст: ('в россии', 'преобразовании')
1/1 [==============================] - 0s 20ms/step
Сгенерированный текст: ('если площадь', 'диагноз')
1/1 [==============================] - 0s 20ms/step
Сгенерированный текст: ('выполнение', 'преобразовании')
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('в современном', 'исследование')
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('хронический', 'ответов')
1/1 [==============================] - 0s 22ms/step
Сгенерированный текст: ('коррекция', 'исследование')


**Однонаправленная однослойная LSTM с использованием keras_nlp.tokenizers.BytePairTokenizer:**

In [ ]:
pip install keras-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 110.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tokenizers import ByteLevelBPETokenizer

# Конвертируем text_corpus в единую строку
text = ' '.join(text_corpus)

# Создаем токенизатор ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([text])

# Конвертируем text_corpus в sequences
sequences = [tokenizer.encode(text).ids for text in text_corpus]

# Pad sequences
padded_sequences = pad_sequences(sequences)

# Готовим данные для обучения
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Создаем модель
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], 1)),
    Dense(tokenizer.get_vocab_size(), activation='softmax')
])

# Компилируем модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Обучаем модель
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
56/56 [==============================] - 2s 6ms/step - loss: 7.9048 - accuracy: 0.3307
Epoch 2/10
56/56 [==============================] - 0s 6ms/step - loss: 5.6641 - accuracy: 0.4080
Epoch 3/10
56/56 [==============================] - 0s 8ms/step - loss: 4.4712 - accuracy: 0.4080
Epoch 4/10
56/56 [==============================] - 0s 8ms/step - loss: 4.2000 - accuracy: 0.4080
Epoch 5/10
56/56 [==============================] - 0s 7ms/step - loss: 4.0562 - accuracy: 0.4086
Epoch 6/10
56/56 [==============================] - 0s 7ms/step - loss: 3.9450 - accuracy: 0.4091
Epoch 7/10
56/56 [==============================] - 0s 8ms/step - loss: 3.8563 - accuracy: 0.4137
Epoch 8/10
56/56 [==============================] - 0s 7ms/step - loss: 3.7880 - accuracy: 0.4368
Epoch 9/10
56/56 [==============================] - 0s 9ms/step - loss: 3.7442 - accuracy: 0.4351
Epoch 10/10
56/56 [==============================] - 0s 8ms/step - loss: 3.7056 - accuracy: 0.4357


In [ ]:
input_text_arr = ["можно получить", "для того чтобы", "в россии", "если площадь", "выполнение", "в современном", "хронический", "коррекция"]
for input_text in input_text_arr:
    input_sequence = tokenizer.encode_batch([input_text])[0]  # Access the first Encoding object
    input_sequence = input_sequence.ids  # Get the token IDs
    input_sequence = np.array([input_sequence])  # Convert to a 2D numpy array
    input_sequence = pad_sequences(input_sequence, maxlen=model.input_shape[1])
    predicted_sequence = model.predict(input_sequence)

    # Decode the predicted sequence
    predicted_sequence = predicted_sequence.argmax(axis=-1)  # Get the token IDs
    predicted_text = tokenizer.decode_batch([predicted_sequence])[0]  # Decode the predicted sequence
    print(f"Сгенерированный текст: {input_text, predicted_text}")

1/1 [==============================] - 1s 1s/step
Сгенерированный текст: ('можно получить', '.')
1/1 [==============================] - 0s 60ms/step
Сгенерированный текст: ('для того чтобы', ' ответов')
1/1 [==============================] - 0s 97ms/step
Сгенерированный текст: ('в россии', '.')
1/1 [==============================] - 0s 70ms/step
Сгенерированный текст: ('если площадь', '.')
1/1 [==============================] - 0s 79ms/step
Сгенерированный текст: ('выполнение', ' ответов')
1/1 [==============================] - 0s 48ms/step
Сгенерированный текст: ('в современном', '.')
1/1 [==============================] - 0s 101ms/step
Сгенерированный текст: ('хронический', '.')
1/1 [==============================] - 0s 44ms/step
Сгенерированный текст: ('коррекция', '.')


# **3. Двунаправленная LSTM**




Выясним наименьшую/наибольшую и среднюю длины строк в нашем массиве предложений

In [ ]:
text_array = filter_non_russian_strings(lines)
# Вычисление максимальной длины строки
max_length = max(len(text) for text in text_array)

# Вычисление минимальной длины строки
min_length = min(len(text) for text in text_array)

# Вычисление средней длины строки
avg_length = sum(len(text) for text in text_array) / len(text_array)

print("Максимальная длина строки:", max_length)
print("Минимальная длина строки:", min_length)
print("Средняя длина строки:", avg_length)

Максимальная длина строки: 1132
Минимальная длина строки: 1
Средняя длина строки: 39.692088671954444


Удаляем тексты слишком длинные/слишком короткие

In [ ]:
# Отфильтруем только строки, которые содержат только русские символы
text_array = filter_non_russian_strings(lines)  
min_length = 35  # Минимальная длина строки, которую нужно сохранить
max_length = 200  # Минимальная длина строки, которую нужно сохранить

text_corpus = [text for text in text_array if len(text) >= min_length and len(text) <= max_length]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Bidirectional, Dense

# Инициализируем токенизатор
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_corpus)

# Преобразуем текст в числовой формат
sequences = tokenizer.texts_to_sequences(text_corpus)

# Задаем максимальную длину последовательности
max_sequence_length = max([len(seq) for seq in sequences])

# Добавляем заполнение для последовательностей разной длины
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Удаляем последний элемент из padded_sequences и используем его в качестве целевой переменной
train_data = padded_sequences[:, :-1]
target_data = padded_sequences[:, 1:]

# Создаем модель
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax'))
])

# Компилируем модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Обучаем модель
model.fit(train_data, target_data, epochs=15, verbose=1)

Epoch 1/15
56/56 [==============================] - 14s 182ms/step - loss: 5.4729 - accuracy: 0.6884
Epoch 2/15
56/56 [==============================] - 6s 112ms/step - loss: 2.4102 - accuracy: 0.7196
Epoch 3/15
56/56 [==============================] - 3s 61ms/step - loss: 2.2530 - accuracy: 0.7237
Epoch 4/15
56/56 [==============================] - 3s 54ms/step - loss: 2.2274 - accuracy: 0.7272
Epoch 5/15
56/56 [==============================] - 3s 57ms/step - loss: 2.2132 - accuracy: 0.7276
Epoch 6/15
56/56 [==============================] - 2s 36ms/step - loss: 2.2012 - accuracy: 0.7276
Epoch 7/15
56/56 [==============================] - 3s 46ms/step - loss: 2.1908 - accuracy: 0.7276
Epoch 8/15
56/56 [==============================] - 2s 29ms/step - loss: 2.1804 - accuracy: 0.7277
Epoch 9/15
56/56 [==============================] - 1s 24ms/step - loss: 2.1680 - accuracy: 0.7279
Epoch 10/15
56/56 [==============================] - 1s 20ms/step - loss: 2.1531 - accuracy: 0.7286
Epoch 

Посмотрим предсказания модели после обучения

In [ ]:
# Генерируем текст
seed_text_arr = ["можно получить", "для того чтобы", "в россии", "если площадь", "выполнение", "в современном", "хронический", "коррекция"]
for seed_text in seed_text_arr:
    seed_sequence = tokenizer.texts_to_sequences([seed_text])
    seed_padded = pad_sequences(seed_sequence, maxlen=max_sequence_length - 1)

    generated_sequence = model.predict(seed_padded, verbose=0)
    generated_indices = np.argmax(generated_sequence, axis=-1)
    generated_text = tokenizer.sequences_to_texts(generated_indices)[0]

    print(f"Сгенерированный текст: {seed_text, generated_text}")

Сгенерированный текст: ('можно получить', 'и в')
Сгенерированный текст: ('для того чтобы', 'в и и и')
Сгенерированный текст: ('в россии', 'в и в')
Сгенерированный текст: ('если площадь', 'в в')
Сгенерированный текст: ('выполнение', 'и')
Сгенерированный текст: ('в современном', 'в и и')
Сгенерированный текст: ('хронический', '')
Сгенерированный текст: ('коррекция', 'в')
